# Wash trades

__DEFINITION__

2 trades filled by a market maker that share a transaction hash where the maker/taker tokens of the first order are the taker/maker tokens of the second

## Prerequisites
1. Install `psycopg2` 
    1. on macos, also need `brew install postgresql` to install
    2. `pip3 install psycopg2`
2. Install `pandas`
3. Have access to a read only user to the database
    1. DM perry for database credentials 

### constants

In [1]:
API_ROOT = "https://api.rook.fi/api/v1/"
MM_ENDPOINT = f"{API_ROOT}coordinator/marketMakers"
AUCTIONS_ENDPOINT = f"{API_ROOT}coordinator/auctions"

# connection constants
PG_USER = ""
PG_PWD = ""
PG_HOST = ""
PG_DB = "staging"

### imports

In [2]:
import math
import psycopg2
import requests
import time

import pandas as pd

from collections import defaultdict

### db connection

In [3]:
conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DB,
    user=PG_USER,
    password=PG_PWD)

In [4]:
cursor = conn.cursor()

### functions

In [5]:
def get_mms():
    response = requests.get(MM_ENDPOINT).json()
    return {mm["name"]: {"addresses": mm["makerAddresses"]} for mm in response}

In [6]:
def get_order_fills(mm_addresses, cursor):
    ofs = []
    cursor.execute("SELECT * FROM public.orderfill")
    for order_fill in cursor:
        txHash = order_fill[0]
        orderHash = order_fill[1]
        maker = order_fill[2]
        makerToken = order_fill[4]
        takerToken = order_fill[5]
        timestamp = order_fill[-1]
#         if maker not in mm_addresses:
#             continue

        
        ofs.append(
            {
                "txHash": txHash,
                "orderHash": orderHash,
                "makerToken": makerToken,
                "takerToken": takerToken,
                "maker": maker,
                "timestamp": timestamp,
            }
        )
            

    return ofs


In [39]:
def get_wash_trades(order_fills):
    txHash_matched_orderFills = defaultdict(list)
    for order_fill in order_fills:
        txHash_matched_orderFills[order_fill["txHash"]].append(order_fill)

    # wash trades have to have at least 2 orderFills
    potential_wash_trades = {
        txHash: orderFills
        for txHash, orderFills in txHash_matched_orderFills.items()
        if len(orderFills) > 1
    }
    
    wash_trades = []
    for txHash, orderFills in potential_wash_trades.items():
        
        if '0x67b1e538685753967976d84221c65bff8aae29390c6a2b938152f83ef334fc32' == txHash:
            print(orderFills)
        
        for i in range(len(orderFills) - 1):
            for j in range(i + 1, len(orderFills)):
                if (
                    orderFills[i]["makerToken"] == orderFills[j]["takerToken"]
                    and orderFills[i]["takerToken"] == orderFills[j]["makerToken"]
                    and orderFills[i]["maker"] == orderFills[j]["maker"]
                ):
                    wash_trades.append(
                        (
                            orderFills[i]["txHash"],
                            orderFills[i]["orderHash"],
                            orderFills[j]["orderHash"],
                            orderFills[i]["maker"],
                            orderFills[i]["timestamp"]
                        )
                    )
    return wash_trades

In [25]:
def get_bid_amounts(order_hashes):
    bids = {order_hash: 0 for order_hash in order_hashes}
    
    for i in range(math.ceil(len(order_hashes) / 30)):
        startIdx = i * 30
        endIdx = (i + 1) * 30
    
        request_params = {"orderHashes": order_hashes[startIdx:endIdx]}
        response = requests.get(AUCTIONS_ENDPOINT, request_params).json()
        
        for auction in response:
            winning_bids = [
                bid["rook_etherUnits"]
                for bid in auction["bidList"]
                if bid["outcome"] and bid["outcome"]["outcomeValue"] > 0
            ]
            if not len(winning_bids):
                continue
            if len(winning_bids) > 1:
                print(f"{auction['orderHash']} has {len(winning_bids)} winning bids")
            bids[auction["orderHash"]] = winning_bids[0]
    return bids

### Execute 

In [26]:
mms = get_mms()
reverse_mms = {
    address: name for name, mm_obj in mms.items() for address in mm_obj["addresses"]
}

In [27]:
addresses = [address for _, mm_obj in mms.items() for address in mm_obj["addresses"]]

In [28]:
order_fills = get_order_fills(addresses, cursor)

In [38]:
wash_trades = get_wash_trades(order_fills)
print(f"Found {len(wash_trades)} potential wash trades")

Is the sus address in potential trades? True
Found 426 potential wash trades


In [30]:
orderHashes = list(
        set(
            [wash_trade[1] for wash_trade in wash_trades]
            + [wash_trade[2] for wash_trade in wash_trades]
        )
    )

In [31]:
bid_amounts = get_bid_amounts(orderHashes)

In [32]:
data = []
for wash_trade in wash_trades:
    (txHash, orderHash1, orderHash2, makerAddress, timestamp) = wash_trade
    maker_name = reverse_mms[makerAddress] if makerAddress in reverse_mms else "N/A"
    bidAmount = bid_amounts[orderHash1]
    bidAmount2 = bid_amounts[orderHash2]
    if bidAmount and not bidAmount2:
        bidAmount2 = bidAmount
    elif bidAmount2 and not bidAmount:
        bidAmount = bidAmount2
    if bidAmount != bidAmount2:
        print(f"bad data for\ntxHash={txHash}, orderHash1={orderHash1}, orderHash2={orderHash2}")
        continue
        
    data.append((maker_name, makerAddress, orderHash1, orderHash2, txHash, bidAmount, timestamp))
washTrade_df = pd.DataFrame(data=data, columns=["MarketMaker", "MakerAddress", "OrderHash1", "OrderHash2", "txHash", "bidAmount", "timestamp"])

    

bad data for
txHash=0xff8bdce5ae3dd758e3bcf6f1203b6a162847630959751b0b064f68ae343b8ad8, orderHash1=0xc54de607ad5dd66fbaf9e0cad29f640522c7f9430c55c64688f8246c3bced106, orderHash2=0x4da42484784d13584f244c05375ecb79cd20a72002bb165727146c16b6b840c6
bad data for
txHash=0x3a6ef97859a27057bd1d551e5dbd9693d5a397b58bfd1f16ec91cf0f97644384, orderHash1=0x67544fa778dad384b2808f393e7547bf9b75748ec403597cd2277f98c843c30f, orderHash2=0x1d904e44123418a874bdcfd8e7f995b3b082fdb94461663c0da1b8c430a20b12
bad data for
txHash=0xefee0546ca25a9266f80ed6ee26ba8544b53243a1c4a7f0720dbe87caa1b94bf, orderHash1=0xc8004aaa324339b4b1c120852f8472ad6456832ba4253b3c5342d594371159af, orderHash2=0x9818d61f9794d045ce5b29d99d381dd2f78b5f393321efca993163b7744cf5e0
bad data for
txHash=0x24c9b86fcf09a74bdbb9eea1f26fea277a3d1ee1879313160718eeba69c48f26, orderHash1=0xa0dec12fb3c65f9b6e6538f49300a8d4a8b7a10c58e3249fac7f29e12a7740c7, orderHash2=0x9818d61f9794d045ce5b29d99d381dd2f78b5f393321efca993163b7744cf5e0
bad data for
txHash=

In [33]:
print(washTrade_df.to_string())

    MarketMaker                                MakerAddress                                                          OrderHash1                                                          OrderHash2                                                              txHash  bidAmount   timestamp
0           N/A  0x15508eb216ba5d44b0988b16c35ffa9bbc1e6f86  0x2338045dd239c15dce59d846cfc17457a74ae30cec918d948877a84b942bc441  0x9301d9592970d6593f57f7bb8cac908b98ae7db2e2cf9c1b82258335a02e0216  0xdf589a4ee1817a0d46752b379756461388c10737c7c722eeff6405f3fbbf2661   0.000000  1628807270
1           N/A  0x60cfc28dece0c0b9ee8f3809f3dfe1e4af2a8a6b  0xdc63f94e540d8fcb716c92e5b9041e44c25ef40a6b852b5ac7b3222418aa96c8  0xf8a213daab7affcfa129f7eded4dfda064b3021d724da9bd5afe2b60bab1fd6a  0x893cff39e7ee300dab3a8e0c2e9cf946df4e1ce06eee2952e82511209b22c263   0.000000  1629073532
2           N/A  0xcd1630a9789ddf0ca780397d74b57a340776e250  0x1a4603c65b8caebb3eca89ac455556b993fa3f9f6ac9c1b9b7a764f926332c4d  0x10dec8b9

In [34]:
num_days = 1
start_time = time.time() - (86400 * num_days)

In [35]:
last_day_wash_trades = washTrade_df.loc[washTrade_df['timestamp'] >= start_time]

In [36]:
print(last_day_wash_trades.to_string())

    MarketMaker                                MakerAddress                                                          OrderHash1                                                          OrderHash2                                                              txHash  bidAmount   timestamp
402         N/A  0xb5ff7be031cea3930b1c6f2afb1b5f459a2a92bb  0x8197e9d68aedc19d3e878359aa403cfc0c5ea4d310702c3d3917b109d1541543  0xa0bba98654f3a4a8ad0eb9f72526213ac9546d0a8d9dc34c402fab1979c6356b  0xd49f84a28543eb6fd8411f3f778faef2cdbf1fbecc2bef93b9c6b0d98a8c0918   0.185784  1654631167
403        C3ll  0x63e8388ac285b27949e9312aab422968e787475f  0xc4f9b59bc503b715c378bf3c87511d5dc6f7f246ec12c87353e60b501d6d6e06  0xa7874a99fbee361fd12464bbe605a1037f413953d82f2f9cb7929666a774a55d  0x8e106ae7a2ef24307fe5c37b1f2f8a8be78185823dc15b6c52ae044148c14b6a   0.129774  1654654163
404  VolleyFire  0x57845987c8c859d52931ee248d8d84ab10532407  0xf163c85d542597546f37897f58fcabd5d8870ea9d73513725970fa8302599fa5  0x4b845462